<a href="https://colab.research.google.com/github/J3SSUS/peruvian-football-news-generator/blob/main/gru/gru_football_news_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRU (Gated Recurrent Unit) - Generator Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
full_text_new = open("/content/drive/MyDrive/nlp/project-nlp-football/full_peruvian_football_news.txt", 'rb').read().decode(encoding='utf-8')

In [ ]:
def replace_tild(text):
    accent_m = {
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'Á': 'A',
        'É': 'E',
        'Í': 'I',
        'Ó': 'O',
        'Ú': 'U'
    }
    for k,v in accent_m.items():
        text = text.replace(k,v)
    return text

In [ ]:
full_text_new = replace_tild(full_text_new)

In [ ]:
print(full_text_new[:2000])

Premio al esfuerzo Piero Quispe fue elegido como el mejor jugador de la Liga 1 2023:
Luego de 10 años de sequía, Universitario formó un gran plantel para gritar campeón este 2023 y una de las piezas fundamentales de este equipo fue Piero Quispe, volante que a sus 23 años fue clave para el elenco dirigido por Jorge Fossati, que se llevó el título ante su clásico rival y este domingo recién pudo recibir el trofeo. El volante crema aportó a su equipo con cuatro goles y dos asistencias, siendo muy influyente en el estilo de juego de Fossati, quien le imprimió un gran juego en colectivo a su equipo, apoyándolo en el mediocampo con jugadores como Rodrigo Ureña o José Pérez Guedes, que eran un respaldo para Quispe. Pierito también destacó en la campaña de Universitario en la Sudamericana, en donde fue elegido en una oportunidad en el once ideal. Estas buenas actuaciones le valieron la convocatoria a la selección nacional dirigida por Juan Reynoso, aunque no ha debutado oficialmente todavía. E

In [ ]:
# The unique characters in the file
vocab = sorted(set(full_text_new))
print(f'{len(vocab)} unique characters')

85 unique characters


In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(full_text_new, 'UTF-8'))
all_ids

<tf.Tensor: shape=(562000,), dtype=int64, numpy=array([36, 64, 51, ...,  1,  2,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

P
r
e
m
i
o
 
a
l
 


In [ ]:
seq_length = 1000
examples_per_epoch = len(full_text_new)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor([b'P' b'r' b'e' ... b'.' b' ' b'E'], shape=(1001,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Premio al esfuerzo Piero Quispe fue elegido como el mejor jugador de la Liga 1 2023:\r\nLuego de 10 a\xc3\xb1os de sequ\xc3\xada, Universitario form\xc3\xb3 un gran plantel para gritar campe\xc3\xb3n este 2023 y una de las piezas fundamentales de este equipo fue Piero Quispe, volante que a sus 23 a\xc3\xb1os fue clave para el elenco dirigido por Jorge Fossati, que se llev\xc3\xb3 el t\xc3\xadtulo ante su cl\xc3\xa1sico rival y este domingo reci\xc3\xa9n pudo recibir el trofeo. El volante crema aport\xc3\xb3 a su equipo con cuatro goles y dos asistencias, siendo muy influyente en el estilo de juego de Fossati, quien le imprimi\xc3\xb3 un gran juego en colectivo a su equipo, apoy\xc3\xa1ndolo en el mediocampo con jugadores como Rodrigo Ure\xc3\xb1a o Jos\xc3\xa9 P\xc3\xa9rez Guedes, que eran un respaldo para Quispe. Pierito tambi\xc3\xa9n destac\xc3\xb3 en la campa\xc3\xb1a de Universitario en la Sudamericana, en donde fue elegido en una oportunidad en el once ideal. Estas buenas actua

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Premio al esfuerzo Piero Quispe fue elegido como el mejor jugador de la Liga 1 2023:\r\nLuego de 10 a\xc3\xb1os de sequ\xc3\xada, Universitario form\xc3\xb3 un gran plantel para gritar campe\xc3\xb3n este 2023 y una de las piezas fundamentales de este equipo fue Piero Quispe, volante que a sus 23 a\xc3\xb1os fue clave para el elenco dirigido por Jorge Fossati, que se llev\xc3\xb3 el t\xc3\xadtulo ante su cl\xc3\xa1sico rival y este domingo reci\xc3\xa9n pudo recibir el trofeo. El volante crema aport\xc3\xb3 a su equipo con cuatro goles y dos asistencias, siendo muy influyente en el estilo de juego de Fossati, quien le imprimi\xc3\xb3 un gran juego en colectivo a su equipo, apoy\xc3\xa1ndolo en el mediocampo con jugadores como Rodrigo Ure\xc3\xb1a o Jos\xc3\xa9 P\xc3\xa9rez Guedes, que eran un respaldo para Quispe. Pierito tambi\xc3\xa9n destac\xc3\xb3 en la campa\xc3\xb1a de Universitario en la Sudamericana, en donde fue elegido en una oportunidad en el once ideal. Estas buen

In [ ]:
# Batch size
BATCH_SIZE = 16

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(16, 1000), dtype=tf.int64, name=None), TensorSpec(shape=(16, 1000), dtype=tf.int64, name=None))>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(16, 1000, 86) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  22016     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  88150     
                                                                 
Total params: 4048470 (15.44 MB)
Trainable params: 4048470 (15.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train the model

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
35/35 [==============================] - 15s 314ms/step - loss: 3.5261
Epoch 2/100
35/35 [==============================] - 12s 321ms/step - loss: 2.6306
Epoch 3/100
35/35 [==============================] - 13s 343ms/step - loss: 2.3618
Epoch 4/100
35/35 [==============================] - 13s 356ms/step - loss: 2.2218
Epoch 5/100
35/35 [==============================] - 12s 341ms/step - loss: 2.0982
Epoch 6/100
35/35 [==============================] - 12s 328ms/step - loss: 1.9649
Epoch 7/100
35/35 [==============================] - 12s 324ms/step - loss: 1.8288
Epoch 8/100
35/35 [==============================] - 12s 327ms/step - loss: 1.7001
Epoch 9/100
35/35 [==============================] - 12s 330ms/step - loss: 1.5862
Epoch 10/100
35/35 [==============================] - 13s 352ms/step - loss: 1.4867
Epoch 11/100
35/35 [==============================] - 13s 349ms/step - loss: 1.4026
Epoch 12/100
35/35 [==============================] - 12s 323ms/step - loss: 1.3316
E

## Inference

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Universitario ganó la final:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Universitario ganó la final:
La festeja de Universitario de Deportes a los 94 minutos del partido y generó la alegría no solo de sus hinchas, sino también del apostador más suertudo del domindo después de 10 años y ante su clásico rival, pero no solo eso, sino también por el jugoso premio que recibirá hoy en el Monumental. Los jugadores de Universitario de Deportes fueron ampayados bebiendo alcohol a altas horas de la madrugada, pero esto fue algo que el administrador crema, Jean Ferrari, minimizó y que, inclusive, deslizó que podría ser para desconcentrarlos de lo deportivo. Esa clase de situaciones ampays la señal de Liga 1 MAX y DirecTV Sports.

¡Alianza va con todo! Barcos Sería lindo ser tricampeón en el Monumental:
Alianza Lima aún no define si comenzará como local o no en las finales por el título de la Liga 1 de esta temporada, pero para Hernán Barcos, ser tricampeón en el Monumental no se compara a nada. Así lo dejó en claro el popular Pirata al ser consultado por los reporter

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Alianza apagó las luces del estadio'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Alianza apagó las luces del estadio Alejandro Villanueva se apagaron poniendo en riesgo a todos los presentes ¿Acaso nunca nos vamos a respetar mutuamente?.

¿Llega a los partidos de la Selección? Zambrano en el Plama para él, y Caraga y su contrato con el Sublete goleada 3-0 a Cantolao en el Alejandro Villanueva 89  Acabó la polemiciar Y GUnio Binacional Diego Ottás la oportunidad de gol para Alianza Lima. Costa remata, pero Barrios ADT despeja. 79  Ocasión clara para ADT que queda invalidada por posición adelantada de Perlaza. 78  Cambio en ADT Sale Pósito y entra Agero. 78  Cambio en Alianza Lima Sale Reyna y entra Soyer. 78  Remate de larga distancia de Mimbela ADT que Campos Alianza Lima ataja fácilmente. 77  ¡Uuuuf! Media vuelta de Barcos Alianza Lima y fuerte remate para intentar abrir el marcador, pero el balón sale del campo de juego. 46  Gran jugada colectiva de Costa 8  FC desde llega, Roda gosparos y cabeza de larga distancia, pero José Carvallo controla sin problemas. 15  

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Universitario se enfrenta a Sporting Cristal el domingo'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Universitario se enfrenta a Sporting Cristal el domingo 16 de abril a las 0400 de la tarde en el estadio Alberto Gallardo por la fecha 4 del Torneo Clausura. Este encuentro será administrado por el árbitro Michael Espinoza en el campo y por el juez Jonathan Zamora interpretando todas las imágenes del partido desde la sala del VAR. La Liga 1 Max será la encargada de hacer llegar la señal del enfrentamiento entre Sporting Cristal y Carlos A. Mannucci a todos los peruanos por medio de los canales 11 y 610 de Bestcable y DirecTV, respectivamente o por la web oficial del torneo.

No se rinden Sporting Cristal volteó el partido y le ganó 1-2 a Sport Huancayo:
A ocho puntos de dicho sería peruano. Así lo h, experiencia y se empató e han seis del campeonato peruano se dejó ver por los hinchas en una fotografía durante el festejo de sus amigos y futbolistas. Como se recuerda, el nacido en Tarapoto dejó a los de Ate a fines del 2016, pero hoy podría volver después de ocho años. Así es, el marcad

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Todo está listo para la final'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Todo está listo para la final de la Liga 1, este emotivo corte audiovisual se ha convertido en un símbolo de la pasión que despierta el fútbol en el Perú y cómo puede unir a las personas en torno a algo más grande que el deporte en sí. Los aficionados de Alianza Lima y Universitario, así como todos los peruanos, se preparan para una final inolvidable, y este video es el preludio perfecto para el partido más emocionante del año.

Alianza Lima recuerda a Comisión Disciplinaria el insulto de Alex Valera a árbitro y pide sanción:
Luego de que Universitario solicite una sanción contra Ángelo Campos, portero de Alianza Lima, salió con una bandera blanquiazul a causa de una lesión. Un esguince en el tobillo, que arrastra hace tres semanas, lo dejaría al margen de la plantilla victoriana, por lo cual el técnico Guillermo Salas estaría pensando en un recambio para la posición de los 3 penieron el mamentán además de su liderate el ataque fue convinún contra un campo del Apertura y, hostaga de re